# Analysis of the returns of bitcoin

## Setup

In [ ]:
import matplotlib.pyplot as plt
from matplotlib.ticker import FuncFormatter
import pandas as pd
import seaborn as sns

In [ ]:
# Get bitcoin price df and set date as index, for more efficiency and simplicity
df_btc = pd.read_csv("../data/BTC.csv", index_col="date", parse_dates=True)

In [ ]:
# Set charts theme
sns.set_theme(style="darkgrid", rc={"grid.alpha": 0.33})
plt.style.use("dark_background")

# Save chart as png function
def save_chart_as_png(filename: str) -> None:
    plt.savefig(
        f"../images/{filename}.png",
        format="png",
        dpi=300,
        orientation="landscape",
        bbox_inches="tight",
    )

In [ ]:
# Get OHLC average as price
df_btc["price"] = df_btc[["open", "high", "low", "close"]].mean(axis=1)
df_btc.drop(columns=["open", "high", "low", "close"], inplace=True)

## Price change year-over-year

In [ ]:
# Get YoY returns
# Get yearly bitcoin price df with first and last prices
df_btc_yearly = df_btc.groupby(df_btc.index.year)["price"].agg(
    first_price="first",
    last_price="last"
)
# Get YoY return
df_btc_yearly["price_change"] = (df_btc_yearly["last_price"] - df_btc_yearly["first_price"]) / df_btc_yearly["first_price"]

In [ ]:
plt.figure(figsize=(10, 6))

sns.barplot(data=df_btc_yearly, x=df_btc_yearly.index, y="price_change", color="#f7931a")

plt.title("Year-over-year return of bitcoin across time")
plt.xlabel("")
plt.ylabel("")

plt.yscale("symlog")
plt.ylim(-1.25, 75)

plt.gca().yaxis.set_major_formatter(FuncFormatter(lambda x, _: f"{int(x)}"))
plt.yticks([-1, 0, 1, 5, 10, 50])

save_chart_as_png("3_BTC_YoY_return")

In [ ]:
# Highest YoY return
df_btc_yearly.loc[[df_btc_yearly["price_change"].idxmax()]]

In [ ]:
# Lowest YoY return
df_btc_yearly.loc[[df_btc_yearly["price_change"].idxmin()]]

In [ ]:
# Average YoY return
df_btc_yearly["price_change"].mean().round(4)

In [ ]:
# Median YoY return
round(df_btc_yearly["price_change"].median(), 4)

In [ ]:
# Standard deviation YoY return
round(df_btc_yearly["price_change"].std(), 4)

In [ ]:
# Cumulative product YoY return
(1 + df_btc_yearly["price_change"]).cumprod().iloc[-1].round(4) - 1

In [ ]:
# Compound annual growth rate (CAGR)
start_value = df_btc["price"].iloc[0]
end_value = df_btc["price"].iloc[-1]
n = (df_btc.index[-1] - df_btc.index[0]).days / 365.25

(end_value / start_value) ** (1 / n) - 1

In [ ]:
# Add positive returns phase column
df_btc_yearly["positive_returns_phase"] = df_btc_yearly.index.map(lambda year:
                                                         1 if 2010 <= year <= 2013
                                                         else 2 if 2015 <= year <= 2017
                                                         else 3 if 2019 <= year <= 2021
                                                         else 4 if 2023 <= year <= 2024
                                                         else None).astype("Int64")

In [ ]:
# Get max, min, average, median, standard deviation and cumulative product YoY return per positive returns phase
df_btc_yearly.groupby("positive_returns_phase").agg(
    timeframe=("price_change", lambda x: f"{x.index.min()}-{x.index.max()}"),
    max_yoy_price_change=("price_change", lambda x: round(x.max(), 2)),
    min_yoy_price_change=("price_change", lambda x: round(x.min(), 2)),
    avg_yoy_price_change=("price_change", lambda x: round(x.mean(), 2)),
    median_yoy_price_change=("price_change", lambda x: round(x.median(), 2)),
    std_yoy_price_change=("price_change", lambda x: round(x.std(), 2)),
    cumprod_yoy_price_change=("price_change", lambda x: round((1 + x).cumprod().iloc[-1] - 1, 2))
)

**Key takeaways:**
- ...

## Risk-Adjusted Performance